## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_DataBase/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ribeira Grande,PT,38.5167,-28.7000,71.55,88,61,11.74,broken clouds
1,1,Kampot,KH,10.6167,104.1833,80.65,90,100,6.44,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,74.26,78,99,15.95,light rain
3,3,East London,ZA,-33.0153,27.9116,63.63,87,63,12.12,broken clouds
4,4,Vanimo,PG,-2.6741,141.3028,77.76,88,100,4.70,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ribeira Grande,PT,38.5167,-28.7000,71.55,88,61,11.74,broken clouds
2,2,Rikitea,PF,-23.1203,-134.9692,74.26,78,99,15.95,light rain
3,3,East London,ZA,-33.0153,27.9116,63.63,87,63,12.12,broken clouds
4,4,Vanimo,PG,-2.6741,141.3028,77.76,88,100,4.70,light rain
5,5,Bethel,US,41.3712,-73.4140,65.08,89,90,0.00,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
643,643,Sibolga,ID,1.7427,98.7792,76.51,85,100,2.21,overcast clouds
644,644,Parkes,AU,-33.1333,148.1833,65.35,41,92,1.01,overcast clouds
646,646,Camacha,PT,33.0833,-16.3333,69.22,73,40,8.05,scattered clouds
650,650,Shintomi,JP,32.0785,131.4723,72.72,67,0,2.35,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                297
City                   297
Country                292
Lat                    297
Lng                    297
Max Temp               297
Humidity               297
Cloudiness             297
Wind Speed             297
Current Description    297
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                292
City                   292
Country                292
Lat                    292
Lng                    292
Max Temp               292
Humidity               292
Cloudiness             292
Wind Speed             292
Current Description    292
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ribeira Grande,PT,71.55,broken clouds,38.5167,-28.7000,
2,Rikitea,PF,74.26,light rain,-23.1203,-134.9692,
3,East London,ZA,63.63,broken clouds,-33.0153,27.9116,
4,Vanimo,PG,77.76,light rain,-2.6741,141.3028,
5,Bethel,US,65.08,overcast clouds,41.3712,-73.4140,
6,Jalu,LY,67.62,clear sky,29.0331,21.5482,
7,Zaraza,VE,76.98,overcast clouds,9.3503,-65.3245,
10,Pisco,PE,64.45,clear sky,-13.7000,-76.2167,
11,Caravelas,BR,73.71,clear sky,-17.7125,-39.2481,
13,Albany,US,61.81,few clouds,42.6001,-73.9662,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
     # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ribeira Grande,PT,71.55,broken clouds,38.5167,-28.7000,Quinta da Meia Eira
2,Rikitea,PF,74.26,light rain,-23.1203,-134.9692,People ThankYou
3,East London,ZA,63.63,broken clouds,-33.0153,27.9116,Tu Casa
4,Vanimo,PG,77.76,light rain,-2.6741,141.3028,Vahmoneh Lodge Vanimo
5,Bethel,US,65.08,overcast clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
...,...,...,...,...,...,...,...
643,Sibolga,ID,76.51,overcast clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
644,Parkes,AU,65.35,overcast clouds,-33.1333,148.1833,Station Motel
646,Camacha,PT,69.22,scattered clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
650,Shintomi,JP,72.72,clear sky,32.0785,131.4723,ホテルスイート


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description<dt><dd>{Current Description} and {Max Temp}°F<dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
current_descrption = hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))